# Delineate Ridge Areas
- delineate the ridge areas from the transformed rasters
- this script will clip, classify, compare, and denoise a raster
- this script is designed to operate on one bend at a time 
- another script is responsible for i/o and matching ridge boundaries to rasters


## Flow within each class
- class will take in and ultimately return file paths for rasters
- the `.execute()` method will perform all major operations for a class including read/write from disk
- all other inidividual methods should just manipulate arrays 


In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import rasterio
import pandas as pd

from scrollstats import BendDataset
from scrollstats.delineation import RasterClipper, BinaryClassifier, RasterAgreementAssessor, RasterDenoiser

## Prepare Inputs

### Set input/output directories

In [ ]:
output_path = Path("data/output")

# Profile Curvature
profc_dir = output_path / "profc"

# Residual Topography
rt_dir = output_path / "rt"

### Filter/sort raster paths

In [ ]:
## regex-like pattern matching to filter dems; 
## leave as "*" to match all files in `dem_dir`
regex_str = "sb_5*.tif"

## function used to sort the dems within `dem_dir`
## simply return `x` to use default filename sorting
def sort_func(x):
    return x

# Create a list of profc paths 
profc_paths = sorted(profc_dir.glob(regex_str), key=sort_func)

# Create a list of rt paths
rt_paths = sorted(rt_dir.glob(regex_str), key=sort_func)

# Create a check to make sure profc and rt paths match up

### Gather bend boundaries

In [ ]:
# Set up bend dataframes
bends = []

for path in profc_paths:
    num = path.stem.split("_")[2]
    bend_id = f"LBR_{num}"
    bend = BendDataset(bend_id).get_bend(proj=True)
    bends.append(bend)
    
bends = pd.concat(bends)
bends

## Process Rasters

### Clip Rasters

In [ ]:
# Clip residual topography rasters
rt_clip_dir = output_path / "rt_clip"

for rt_path, bend in zip(rt_paths, bends["geometry"]):
    rc = RasterClipper(rt_path, bend, rt_clip_dir)
    clip_path = rc.execute()

################################
# Plot results

rt = rasterio.open(rt_path).read(1)
rt_clip = rasterio.open(clip_path).read(1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

mapper = ax1.imshow(rt)
ax1.set_axis_off()
ax1.set_title("Original Raster")


mapper = ax2.imshow(rt_clip)
ax2.set_axis_off()
ax2.set_title("Clipped Raster");


In [ ]:
# Clip profile curvature rasters
profc_clip_dir = output_path / "profc_clip"

for profc_path, bend in zip(profc_paths, bends["geometry"]):
    rc = RasterClipper(profc_path, bend, profc_clip_dir)
    clip_path = rc.execute()


################################
# Plot results

profc = rasterio.open(profc_path).read(1)
profc_clip = rasterio.open(clip_path).read(1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

mapper = ax1.imshow(profc)
ax1.set_axis_off()
ax1.set_title("Original Raster")

mapper = ax2.imshow(profc_clip)
ax2.set_axis_off()
ax2.set_title("Clipped Raster");

### Create binary classification


In [ ]:
# Classify Profile Curvature raster
profc_clip_paths = sorted(profc_clip_dir.glob(regex_str), key=sort_func)
profc_bin_dir = output_path / "profc_bin"

for path in profc_clip_paths:
    bc = BinaryClassifier(path, 0, profc_bin_dir)
    out_path = bc.execute()


################################
# Plot results

profc = rasterio.open(path).read(1)
profc_bin = rasterio.open(out_path).read(1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

mapper = ax1.imshow(profc)
ax1.set_axis_off()
ax1.set_title("Original Raster")

mapper = ax2.imshow(profc_bin)
ax2.set_axis_off()
ax2.set_title("Binary Raster");

In [ ]:
rt_clip_paths = sorted(rt_clip_dir.glob(regex_str), key=sort_func)
rt_bin_dir = output_path / "rt_bin"

for path in rt_clip_paths:
    bc = BinaryClassifier(path, 0, rt_bin_dir)
    out_path = bc.execute()


################################
# Plot results

rt = rasterio.open(path).read(1)
rt_bin = rasterio.open(out_path).read(1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

mapper = ax1.imshow(rt)
ax1.set_axis_off()
ax1.set_title("Original Raster")

mapper = ax2.imshow(rt_bin)
ax2.set_axis_off()
ax2.set_title("Binary Raster");

### Test Agreement

In [ ]:
rt = rt_clip_paths[0].stem
profc = profc_clip_paths[0].stem

bend_id = rt[:rt.find("_rt")]

In [ ]:
profc_bin_paths = sorted(profc_bin_dir.glob(regex_str), key=sort_func)
rt_bin_paths = sorted(rt_bin_dir.glob(regex_str), key=sort_func)
bend_ids = [p.stem[:p.stem.find("_profc")] for p in profc_bin_paths]


for (profc, rt, bend_id) in zip(profc_bin_paths, rt_bin_paths, bend_ids):
    ra = RasterAgreementAssessor(profc, rt, bend_id, output_path)
    out_path = ra.execute()


################################
# Plot results

profc = rasterio.open(profc).read(1)
rt = rasterio.open(rt).read(1)
agr = rasterio.open(out_path).read(1)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 6))

mapper = ax1.imshow(profc, alpha=0.7)
ax1.set_axis_off()
ax1.set_title("Profile Curvature")

mapper = ax2.imshow(agr)
ax2.set_axis_off()
ax2.set_title("Agreement")

mapper = ax3.imshow(rt, alpha=0.7)
ax3.set_axis_off()
ax3.set_title("Residual Topography");

### Test Denoise

In [ ]:
# Set thresholds for denoising
small_feats_size = 500
elongation_threshold = 80

# Set input dir
agr_dir = output_path / "agr"

# Set out_dir
denoise_dir = output_path / "agr_denoise"

In [ ]:
agr_paths = sorted(agr_dir.glob(regex_str), key=sort_func)

for path in agr_paths:
    dn = RasterDenoiser(path, small_feats_size, elongation_threshold, denoise_dir)
    out_path = dn.execute()

################################
# Plot results
agr = rasterio.open(path).read(1)
denoise = rasterio.open(out_path).read(1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

mapper = ax1.imshow(agr)
ax1.set_axis_off()
ax1.set_title("Agreement")

mapper = ax2.imshow(denoise)
ax2.set_axis_off()
ax2.set_title("Denoised");